In [163]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time

In [164]:
links = ['https://www.cmegroup.com/markets/agriculture/oilseeds/soybean.volume.html','https://www.cmegroup.com/markets/agriculture/oilseeds/soybean-meal.volume.html','https://www.cmegroup.com/markets/agriculture/oilseeds/soybean-oil.volume.html','https://www.cmegroup.com/markets/agriculture/oilseeds/soybean-oil.volume.html','https://www.cmegroup.com/markets/agriculture/grains/wheat.volume.html','https://www.cmegroup.com/markets/agriculture/grains/wheat.volume.html']
products = ['Soybean','Soymeal','Soyoil','Corn','SRW','HRW']
x = 4

In [165]:

# # Set options for Microsoft Edge
# options = Options()
# options.use_chromium = True  # Use this option if using Edge Chromium

# # Specify the path to the WebDriver executable
# webdriver_path = r"C:\Users\anmol.chopra\Downloads\edgedriver_win64 (1)\msedgedriver.exe"

# # Initialize the WebDriver
# service = Service(webdriver_path)
# driver = webdriver.Edge(service=service, options=options)

# # Open a particular website
# url = links[x]
# driver.get(url)

# time.sleep(10)

# html_source = driver.page_source

# soup = BeautifulSoup(html_source, 'html.parser')

# # Locate all tables
# tables = soup.find_all('div', {'class': 'main-table-wrapper'})


# # Check if there are enough tables
# if len(tables) > 1:
#     table = tables[1].find('table')  # Select the second table
# else:
#     raise ValueError("The second table is not found on the page.")


# label_element = driver.find_element(By.XPATH, '//label[text()="Trade Date"]')

# # Find the adjacent sibling element containing the date
# date_element = label_element.find_element(By.XPATH, './following-sibling::div//span[@class="button-text"]')

# # Extract the date text from the element
# date_text = date_element.text

# # Extract table headers
# headers = []
# header_rows = table.find_all('thead')
# for header_row in header_rows:
#     cols = header_row.find_all('th')
#     for col in cols:
#         colspan = int(col.get('colspan', 1))
#         headers.extend([col.get_text(separator=' ').strip()] * colspan)

# # Extract table rows
# rows = []
# for row in table.find_all('tbody')[0].find_all('tr'):
#     cols = row.find_all('td')
#     if len(cols) > 0:
#         cols = [ele.get_text().strip() for ele in cols]
#         rows.append(cols)

# # Create DataFrame
# df = pd.DataFrame(rows, columns=headers[:len(rows[0])])

# # Print DataFrame
# print(df)

# driver.quit()


In [166]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup

# Set options for Microsoft Edge
options = Options()
options.use_chromium = True  # Use this option if using Edge Chromium

# Specify the path to the WebDriver executable
webdriver_path = r"C:\Users\anmol.chopra\Downloads\edgedriver_win64 (1)\msedgedriver.exe"

# Initialize the WebDriver
service = Service(webdriver_path)
driver = webdriver.Edge(service=service, options=options)

# Open a particular website
url = links[x]
driver.get(url)

time.sleep(10)

html_source = driver.page_source

soup = BeautifulSoup(html_source, 'html.parser')

# Locate all tables
tables = soup.find_all('div', {'class': 'main-table-wrapper'})

# Check if there are enough tables
if len(tables) > 1:
    table1 = tables[0].find('table')  # Select the first table
    table2 = tables[1].find('table')  # Select the second table
else:
    raise ValueError("Not enough tables found on the page.")


td_elements = table1.find_all('td')

# Extract the text from each <td> element and remove commas
values = [int(td.get_text().replace(',', '')) for td in td_elements]

print(values)

# Extract table headers
headers = []
header_rows = table2.find_all('thead')
for header_row in header_rows:
    cols = header_row.find_all('th')
    for col in cols:
        colspan = int(col.get('colspan', 1))
        headers.extend([col.get_text(separator=' ').strip()] * colspan)

# Extract table rows
rows = []
for row in table2.find_all('tbody')[0].find_all('tr'):
    cols = row.find_all('td')
    if len(cols) > 0:
        cols = [ele.get_text().strip() for ele in cols]
        rows.append(cols)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers[:len(rows[0])])

# Print DataFrame
print(df)

# Extract Trade Date
label_element = driver.find_element(By.XPATH, '//label[text()="Trade Date"]')

# Find the adjacent sibling element containing the date
date_element = label_element.find_element(By.XPATH, './following-sibling::div//span[@class="button-text"]')

# Extract the date text from the element
date_text = date_element.text
print("\nTrade Date:", date_text)

driver.quit()


[165969, 0, 1430, 167399, 132, 596, 2, 21857, 0, 422894, -8653]
      Month  Volume Volume Volume  Volume Volume Volume Volume Volume  \
0  SEP 2024  79,518      0    393  79,911      0    391      2  9,548   
1  DEC 2024  61,151      0    687  61,838    132    205      0  9,979   
2  MAR 2025  16,964      0      0  16,964      0      0      0  1,359   
3  MAY 2025   5,176      0    350   5,526      0      0      0    501   
4  JUL 2025   2,801      0      0   2,801      0      0      0    470   
5  SEP 2025     227      0      0     227      0      0      0      0   
6  DEC 2025     128      0      0     128      0      0      0      0   
7  MAR 2026       4      0      0       4      0      0      0      0   
8  MAY 2026       0      0      0       0      0      0      0      0   
9  JUL 2026       0      0      0       0      0      0      0      0   

  Deliveries Open Interest Open Interest  
0          0       185,928       -10,888  
1          0       142,064          -637  
2  

In [167]:
df = pd.DataFrame(rows, columns=headers[:len(rows[0])])
df = df[['Month','Open Interest']]
df.columns = ['Month','OI','Open Interest']
df = df.drop(columns='OI')
df

,Month,Open Interest
0,SEP 2024,"-10,888"
1,DEC 2024,-637
2,MAR 2025,"+1,465"
3,MAY 2025,+998
4,JUL 2025,+401
5,SEP 2025,-9
6,DEC 2025,+17
7,MAR 2026,0
8,MAY 2026,0
9,JUL 2026,0


In [168]:
df_new = df.T
df_new = df_new.astype(str)
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(df_new.index[0])
df_new.insert(0,'Date',date_text.split(', ')[1] )
# df_new.to_excel('OI Data.xlsx',index=False)
# df_new.to_csv('OI Data.csv',index=False)

df_new

Month,Date,SEP 2024,DEC 2024,MAR 2025,MAY 2025,JUL 2025,SEP 2025,DEC 2025,MAR 2026,MAY 2026,JUL 2026
Open Interest,07 Aug 2024,"-10,888",-637,"+1,465",+998,+401,-9,+17,0,0,0


In [169]:
df_new['Market Open Interest'] = values[-2]
df_new['Change in OI'] = values[-1]

In [172]:
values[-2:]

[422894, -8653]

In [171]:
df_new.to_csv('OI Data-'+products[x]+'.csv',index=False)

In [157]:
import pandas as pd
from openpyxl import Workbook,load_workbook


df1 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Corn.csv')
df2 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-HRW.csv')
df3 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soybean.csv')
df4 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soymeal.csv')
df5 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soyoil.csv')
df6 = pd.read_csv(r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-SRW.csv')

file_path = r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data.xlsx'

wb = Workbook()

wb.save(file_path)

# Save dataframes to different sheets in the cleared Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    df1.to_excel(writer, sheet_name='Corn', index=False)
    df2.to_excel(writer, sheet_name='HRW', index=False)
    df3.to_excel(writer, sheet_name='Soybean', index=False)
    df4.to_excel(writer, sheet_name='Soymeal', index=False)
    df5.to_excel(writer, sheet_name='Soyoil', index=False)
    df6.to_excel(writer, sheet_name='SRW', index=False)


In [162]:
# File paths
file_path = r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data.xlsx'

csv_paths = [
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Corn.csv',
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-HRW.csv',
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soybean.csv',
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soymeal.csv',
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-Soyoil.csv',
    r'\\corp.hertshtengroup.com\Users\India\Data\anmol.chopra\Documents\FF Codes\Futures-First\OI\OI Data-SRW.csv'
]

sheet_names = ['Corn', 'HRW', 'Soybean', 'Soymeal', 'Soyoil', 'SRW']

# Load existing Excel workbook
wb = load_workbook(file_path)

# Iterate through each CSV file and corresponding sheet name
for csv_path, sheet_name in zip(csv_paths, sheet_names):
    # Read the last row of the CSV file
    df = pd.read_csv(csv_path).iloc[[-1]]
    
    # Load the sheet to append the last row
    if sheet_name in wb.sheetnames:
        with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False, header=False, startrow=writer.sheets[sheet_name].max_row)

# Save the workbook
wb.save(file_path)